In [4]:
from dotenv import load_dotenv
import os

# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])
print(OPENAI_API_KEY[:5])

5L
sk-pr


#### 문제 3-1 :  콘텐츠분쟁해결 RAG 시스템

In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_upstage import ChatUpstage
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_upstage import UpstageEmbeddings

loader = PyPDFLoader('../data/콘텐츠분쟁해결_사례.pdf')
documents = loader.load()

In [8]:
def main():
    """
    콘텐츠 분쟁 해결 사례집을 기반으로 법률 자문을 제공하는 RAG 시스템을 구축하고 실행합니다.
    """
    # .env 파일에서 환경 변수 로드
    load_dotenv()

    # UPSTAGE_API_KEY 환경 변수 확인
    if "UPSTAGE_API_KEY" not in os.environ:
        print("에러: UPSTAGE_API_KEY 환경 변수가 설정되지 않았습니다.")
        print("'.env' 파일에 UPSTAGE_API_KEY='your_api_key' 형식으로 키를 추가해주세요.")
        return

    print("="*50)
    print("콘텐츠 분쟁 해결 RAG 시스템 - 실습 시작")
    print("="*50)

    # # 0단계: 문서 로드
    # # PDF 파일을 로드합니다. 파일 경로는 실제 위치에 맞게 수정해야 할 수 있습니다.
    # try:
    #     loader = PyPDFLoader('./콘텐츠분쟁해결_사례.pdf')
    #     documents = loader.load()
    #     print(f"\n[0단계: 문서 로드] 성공: 총 {len(documents)} 페이지")
    # except Exception as e:
    #     print(f"\n[0단계: 문서 로드] 실패: PDF 파일을 찾을 수 없거나 로드할 수 없습니다.")
    #     print("  - 현재 디렉토리에 '콘텐츠분쟁해결_사례.pdf' 파일이 있는지 확인하세요.")
    #     print(f"  - 상세 오류: {e}")
    #     return

    # 1단계: 문서 분할 설정
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=300,
        separators=[
            "\n【사건개요】",
            "\n【쟁점사항】",
            "\n【처리경위】",
            "\n【처리결과】",
            "\n■", "\n\n", "\n", ".", " ", ""
        ]
    )
    docs = text_splitter.split_documents(documents)
    print(f"[1단계: 문서 분할] 완료: {len(docs)}개의 문서 조각(chunk) 생성")

    # 2단계: 임베딩 모델 설정
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    print("[2단계: 임베딩 모델] 설정 완료: solar-embedding-1-large")

    # 벡터 스토어 생성 (FAISS 사용)
    # 로드된 문서 조각들을 임베딩하여 FAISS 벡터 스토어에 저장합니다.
    try:
        vectorstore = FAISS.from_documents(docs, embeddings)
        print("[벡터 스토어] 생성 완료 (FAISS)")
    except Exception as e:
        print(f"[벡터 스토어] 생성 실패: {e}")
        return


    # 3단계: 검색기(Retriever) 설정
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5}
    )
    print("[3단계: 검색기 설정] 완료: 유사도 기반, 상위 5개 결과 검색")

    # 4단계: LLM 설정
    llm = ChatUpstage(
        model="solar-pro",
        temperature=0.5
    )
    print("[4단계: LLM 설정] 완료: solar-1-mini (temperature=0.2)")


    # 5단계: 법률 자문 프롬프트 작성
    prompt_template = """
    당신은 콘텐츠 분야 전문 법률 자문가입니다.
    아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

    관련 분쟁사례:
    {context}

    상담 내용: {question}

    답변 가이드라인:
    1. 사례를 근거로 답변하세요.
    2. 관련 법령을 명시하세요.  
    3. 단계별 해결방안을 제시하세요.
    4. 유사 사례를 참조하세요.
    5. 없는 정보는 "확인할 수 없습니다"라고 하세요.


    전문 법률 조언:"""

    prompt = PromptTemplate.from_template(prompt_template)
    print("[5단계: 프롬프트 작성] 완료: 법률 자문 프롬프트가 설정되었습니다.")

    # 6단계: QA 체인 생성
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )
    print("[6단계: QA 체인 생성] 완료: RAG 파이프라인이 준비되었습니다.")

    # 7단계: 테스트 질문 작성
    test_questions = [
        "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
        "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
        "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
        "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
        "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
    ]
    print(f"\n[7단계: 테스트 질문] {len(test_questions)}개의 테스트 질문이 준비되었습니다.")

    # 8단계: 분쟁 유형 분류 함수 (선택)
    def classify_dispute_type(query):
        game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
        elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
        web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]
        
        query_lower = query.lower()
        
        if any(keyword in query_lower for keyword in game_keywords):
            return "게임"
        elif any(keyword in query_lower for keyword in elearning_keywords):
            return "이러닝"
        elif any(keyword in query_lower for keyword in web_keywords):
            return "웹콘텐츠"
        else:
            return "기타"
    print("[8단계: 분류 함수] 분쟁 유형 분류 함수가 정의되었습니다.")

    # 테스트 질문 실행
    print("\n" + "="*50)
    print("테스트 질문에 대한 법률 자문을 시작합니다.")
    print("="*50 + "\n")

    for i, question in enumerate(test_questions, 1):
        print(f"--- 질문 {i} ---")
        
        # 8단계 함수 사용 예시
        dispute_type = classify_dispute_type(question)
        print(f"상담 내용: {question}")
        print(f"(분류된 분쟁 유형: {dispute_type})")
        print("\n[AI 법률 자문가 답변]")
        print("답변 생성 중...")

        try:
            # QA 체인 실행
            result = qa_chain.invoke({"query": question})
            
            # 결과 출력
            print(result["result"].strip())
            
            # 참조한 소스 문서 출력
            source_docs = result.get("source_documents", [])
            if source_docs:
                print("\n[참조한 사례 정보]")
                for doc_num, doc in enumerate(source_docs, 1):
                    # page_content의 첫 100자만 미리보기로 표시
                    content_preview = doc.page_content.strip().replace('\n', ' ')[:100]
                    page_number = doc.metadata.get('page', 'N/A')
                    print(f"  - 문서 {doc_num} (Page: {page_number}): \"{content_preview}...\"")

        except Exception as e:
            print(f"  오류 발생: 답변을 생성하는 동안 문제가 발생했습니다.")
            print(f"  상세 오류: {e}")

        print("\n" + "-"*20 + "\n")

if __name__ == '__main__':
    main()

콘텐츠 분쟁 해결 RAG 시스템 - 실습 시작
[1단계: 문서 분할] 완료: 104개의 문서 조각(chunk) 생성
[2단계: 임베딩 모델] 설정 완료: solar-embedding-1-large
[벡터 스토어] 생성 완료 (FAISS)
[3단계: 검색기 설정] 완료: 유사도 기반, 상위 5개 결과 검색
[4단계: LLM 설정] 완료: solar-1-mini (temperature=0.2)
[5단계: 프롬프트 작성] 완료: 법률 자문 프롬프트가 설정되었습니다.
[6단계: QA 체인 생성] 완료: RAG 파이프라인이 준비되었습니다.

[7단계: 테스트 질문] 5개의 테스트 질문이 준비되었습니다.
[8단계: 분류 함수] 분쟁 유형 분류 함수가 정의되었습니다.

테스트 질문에 대한 법률 자문을 시작합니다.

--- 질문 1 ---
상담 내용: 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
(분류된 분쟁 유형: 게임)

[AI 법률 자문가 답변]
답변 생성 중...
### 전문 법률 조언: 온라인 게임 시스템 오류로 인한 아이템 복구 거부 시 해결방안

#### 1. **사례 분석 및 법적 근거**  
제공된 분쟁조정 사례를 종합하면, 시스템 오류로 인한 아이템 복구 문제는 다음 요소에 따라 해결 여부가 결정됩니다:  
- **(1) 시스템 오류의 객관적 입증**: 게임사의 기술적 결함이 확인되어야 함 (2009년 사례 참조).  
- **(2) 계정 소유권**: 명의자와 실소유자가 일치해야 복구 가능성 높음 (2006년 사례 참조).  
- **(3) 게임사의 약관 조항**: 대부분의 게임사는 "아이템의 소유권 귀속" 및 "복구 불가" 조항을 약관에 명시함.  

**관련 법령**:  
- **민법 제250조(도품·유실물 특례)**: 아이템이 "금전" 또는 "디지털 콘텐츠"로 해석될 경우 적용되지 않을 수 있음.  
- **전자상거래법 제17조(청약철회 등)**: 디지털 콘텐츠는 제공이 시작된 경